# Processing data for modelling

## - Stock price

In [1]:
import pandas as pd
from tqdm import tqdm

from functions import print_shape


In [2]:
# test with feature_engineering.py
prices = pd.read_csv('data/train_files/stock_prices.csv', parse_dates=[1])
print_shape(prices)
prices.head(2)

 Shape:
 ----------------------------------------
 Observations:   2.33M
 Features:       12
 Feature Date:    datetime64[ns]
----------------------------------------
Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       'Target'],
      dtype='object')
----------------------------------------
RowId missing Values: 0 (0.0%)
Date missing Values: 0 (0.0%)
SecuritiesCode missing Values: 0 (0.0%)
Open missing Values: 7608 (0.3%)
High missing Values: 7608 (0.3%)
Low missing Values: 7608 (0.3%)
Close missing Values: 7608 (0.3%)
Volume missing Values: 0 (0.0%)
AdjustmentFactor missing Values: 0 (0.0%)
ExpectedDividend missing Values: 2313666 (99.2%)
SupervisionFlag missing Values: 0 (0.0%)
Target missing Values: 238 (0.0%)


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324


In [3]:

from feature_engineering import fill_and_drop_na_values

fill_prices = fill_and_drop_na_values(prices, drop=False)
fill_prices.to_csv('data/curr_fill_prices.csv')
print_shape(fill_prices)
fill_prices.head(2)
del fill_prices


100%|██████████| 2000/2000 [03:29<00:00,  9.52it/s]


 Shape:
 ----------------------------------------
 Observations:   2.33M
 Features:       12
 Feature Date:    datetime64[ns]
----------------------------------------
Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       'Target'],
      dtype='object')
----------------------------------------
RowId missing Values: 0 (0.0%)
Date missing Values: 0 (0.0%)
SecuritiesCode missing Values: 0 (0.0%)
Open missing Values: 264 (0.0%)
High missing Values: 264 (0.0%)
Low missing Values: 264 (0.0%)
Close missing Values: 264 (0.0%)
Volume missing Values: 0 (0.0%)
AdjustmentFactor missing Values: 0 (0.0%)
ExpectedDividend missing Values: 127359 (5.5%)
SupervisionFlag missing Values: 0 (0.0%)
Target missing Values: 238 (0.0%)


In [4]:
fill_prices = pd.read_csv('data/curr_fill_prices.csv', parse_dates=['Date'], index_col=[0])
print_shape(fill_prices)

 Shape:
 ----------------------------------------
 Observations:   2.33M
 Features:       12
 Feature Date:    datetime64[ns]
----------------------------------------
Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       'Target'],
      dtype='object')
----------------------------------------
RowId missing Values: 0 (0.0%)
Date missing Values: 0 (0.0%)
SecuritiesCode missing Values: 0 (0.0%)
Open missing Values: 264 (0.0%)
High missing Values: 264 (0.0%)
Low missing Values: 264 (0.0%)
Close missing Values: 264 (0.0%)
Volume missing Values: 0 (0.0%)
AdjustmentFactor missing Values: 0 (0.0%)
ExpectedDividend missing Values: 127359 (5.5%)
SupervisionFlag missing Values: 0 (0.0%)
Target missing Values: 238 (0.0%)


In [5]:
from feature_engineering import adjust_price


ad_price = adjust_price(fill_prices)
ad_price.to_csv('data/curr_ad_price.csv')
print_shape(ad_price)
ad_price.head(2)
del ad_price

100%|██████████| 2000/2000 [03:52<00:00,  8.61it/s]


 Shape:
 ----------------------------------------
 Observations:   2.33M
 Features:       13
 Feature Date:    datetime64[ns]
----------------------------------------
Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       'Target', 'ad_Target'],
      dtype='object')
----------------------------------------
RowId missing Values: 0 (0.0%)
Date missing Values: 0 (0.0%)
SecuritiesCode missing Values: 0 (0.0%)
Open missing Values: 264 (0.0%)
High missing Values: 264 (0.0%)
Low missing Values: 264 (0.0%)
Close missing Values: 264 (0.0%)
Volume missing Values: 0 (0.0%)
AdjustmentFactor missing Values: 0 (0.0%)
ExpectedDividend missing Values: 127359 (5.5%)
SupervisionFlag missing Values: 0 (0.0%)
Target missing Values: 238 (0.0%)
ad_Target missing Values: 4238 (0.2%)


In [6]:
ad_price = pd.read_csv('data/curr_ad_price.csv', parse_dates=['Date'], index_col=[0])
print_shape(ad_price)

 Shape:
 ----------------------------------------
 Observations:   2.33M
 Features:       13
 Feature Date:    datetime64[ns]
----------------------------------------
Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       'Target', 'ad_Target'],
      dtype='object')
----------------------------------------
RowId missing Values: 0 (0.0%)
Date missing Values: 0 (0.0%)
SecuritiesCode missing Values: 0 (0.0%)
Open missing Values: 264 (0.0%)
High missing Values: 264 (0.0%)
Low missing Values: 264 (0.0%)
Close missing Values: 264 (0.0%)
Volume missing Values: 0 (0.0%)
AdjustmentFactor missing Values: 0 (0.0%)
ExpectedDividend missing Values: 127359 (5.5%)
SupervisionFlag missing Values: 0 (0.0%)
Target missing Values: 238 (0.0%)
ad_Target missing Values: 4238 (0.2%)


In [8]:
from feature_engineering import price_new_features, encode_flag

ad_price_feat = price_new_features(ad_price)

ad_price_feat['SupervisionFlag'] = encode_flag(ad_price_feat)
ad_price_feat.to_csv('data/curr_ad_price_feat.csv')
print_shape(ad_price_feat)
del ad_price_feat

  0%|          | 0/2000 [00:00<?, ?it/s]


KeyError: 'ad_Close'

In [ ]:
ad_price_feat =  pd.read_csv('data/curr_ad_price_feat.csv', parse_dates=['Date'], index_col=[0])
print_shape(ad_price_feat)

In [ ]:
ad_price_feat.columns

## - Financials

In [ ]:
#from feature_engineering import fill_finances

financial = financial = pd.read_csv('data/train_files/financials.csv',parse_dates=['Date'])
financial.head(2)

In [ ]:
from feature_engineering import fill_finances_knn



filled_finances = fill_finances_knn(financial, prices)
filled_finances.to_csv('data/curr_filled_finances.csv')
print_shape(filled_finances)
filled_finances.head()
del filled_finances

In [ ]:
filled_finances = pd.read_csv('data/curr_filled_finances.csv', parse_dates=['Date'], index_col=[0])
print_shape(filled_finances)

In [ ]:
# ffill and drop 


"""def new_features_financial(filled_finances):
    sec_codes = filled_finances.SecuritiesCode.unique()

    filled_financial_feat = pd.DataFrame(columns=filled_finances.columns)

    for i in tqdm(sec_codes):
        # select a security code
        aktie = filled_finances.query('SecuritiesCode == @i')
        aktie.sort_values('Date')
        # create new features:
        aktie['margin'] = aktie['Profit'] / aktie['NetSales'] * 100
        aktie['profit_ttm'] = aktie['Profit'].shift(3) + aktie['Profit'].shift(2) + aktie['Profit'].shift(1) + aktie['Profit']
        aktie['rev_ttm'] = aktie['NetSales'].shift(3) + aktie['NetSales'].shift(2) + aktie['NetSales'].shift(1) + aktie['NetSales']
        aktie['win_quarter_growth'] = (aktie['Profit'] - aktie['Profit'].shift(1)) / aktie['Profit'].shift(1) * 100
        aktie['rev_quarter_growth'] = (aktie['NetSales'] - aktie['NetSales'].shift(1)) / aktie['NetSales'].shift(1) * 100
        aktie['win_yoy_growth'] = (aktie['Profit'] - aktie['Profit'].shift(4)) / aktie['Profit'].shift(4) * 100
        aktie['rev_yoy_growth'] = (aktie['NetSales'] - aktie['NetSales'].shift(4)) / aktie['NetSales'].shift(4) * 100
        aktie['win_ttm_growth'] = (aktie['profit_ttm'] - aktie['profit_ttm'].shift(1)) / aktie['profit_ttm'].shift(1) * 100
        aktie['rev_ttm_growth'] = (aktie['rev_ttm'] - aktie['rev_ttm'].shift(1)) / aktie['rev_ttm'].shift(1) * 100
        aktie['margin_growth'] = (aktie['margin'] - aktie['margin'].shift()) / aktie['margin'].shift() * 100
        
        # fill
        aktie = aktie.ffill()
        #aktie = aktie.dropna(axis=0)

        filled_financial_feat  = pd.concat([filled_financial_feat , aktie])

        filled_financial_feat['Date'] = pd.to_datetime(filled_financial_feat['Date']) 
    
    return filled_financial_feat"""

In [ ]:
from feature_engineering import new_features_financial

filled_financial_features = new_features_financial(filled_finances)
filled_financial_features.to_csv('data/curr_filled_financial_features.csv')
print_shape(filled_financial_features)
filled_financial_features.head()



In [ ]:
missingValues(filled_financial_features)

In [ ]:
filled_financial_features.SecuritiesCode = filled_financial_features.SecuritiesCode.astype(int)

In [ ]:
filled_financial_features.head(2)

In [ ]:
ad_price.head(2)

## - financials + price

In [ ]:
# create key on financial : RowId
filled_financial_features['RowId'] = filled_financial_features.Date.dt.strftime('%Y%m%d').astype(str) + '_' + filled_financial_features.SecuritiesCode.astype(str)

In [ ]:
filled_financial_features.sort_values('Date').head(20)

In [ ]:
filled_financial_features.RowId.info()

In [ ]:
# concat financials and price
price_financial = pd.merge(ad_price, filled_financial_features, how='left', on='RowId', suffixes=[None, 'f_'])
#price_financial = pd.concat([ad_price, filled_financial_features], keys='RowID')

In [ ]:
print_shape(price_financial)

In [ ]:
price_financial.head()

In [ ]:
price_financial.columns


In [ ]:
from functions import missingValues

missingValues(price_financial)

In [ ]:


"""def ffill_pro_code(df):
    sec_codes = df.SecuritiesCode.unique()

    df_fill_na = pd.DataFrame(columns=price_financial.columns)

    for i in tqdm(sec_codes):

        current = df.query('SecuritiesCode == @i').sort_values('Date')

        current = current.ffill()
        #current = current.dropna(axis=0)

        df_fill_na  = pd.concat([df_fill_na , current])

    df_fill_na['Date'] = pd.to_datetime(df_fill_na['Date'])
    
    return df_fill_na"""

In [ ]:
#price_financial_fill = ffill_pro_code(price_financial)
price_financial_fill = fill_and_drop_na_values(price_financial, dropna=False)


In [ ]:
print_shape(price_financial_fill)

In [ ]:
# new features
#price_financial_fill.drop_duplicates(inplace=True)
missingValues(filled_financial_features)

In [ ]:
# save dataframe to csv
price_financial_fill.to_csv('data/curr_filled_financial_features.csv')

In [ ]:
print_shape(price_financial_fill)